In [1]:
import math
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
from config import *
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

## Import

In [2]:
df_douban_inter = pd.read_csv(data_preprocessing + 'douban_inter.csv', dtype={'user_id': str, 'item_id': str, 'rating': int})
df_douban_user_feats = pd.read_csv(data_preprocessing + 'douban_user_features.csv', dtype=str)
df_douban_item_feats = pd.read_csv(data_preprocessing + 'douban_item_features.csv', dtype={'item_id': str, 'author': str, 'publisher': str, 'year': str})
# df_douban_inter

In [3]:
df_movie_inter = pd.read_csv(data_preprocessing + 'movie_inter.csv', dtype={'user_id': str, 'item_id': str, 'rating': int, 'timestamp': int})
df_movie_user_feats = pd.read_csv(data_preprocessing + 'movie_user_features.csv', dtype=str)
df_movie_item_feats = pd.read_csv(data_preprocessing + 'movie_item_features.csv', dtype=str)
# df_movie_inter

In [4]:
df_yelp_inter = pd.read_csv(data_preprocessing + 'yelp_inter.csv', dtype={'user_id': str, 'item_id': str, 'rating': int})
df_yelp_user_feats = pd.read_csv(data_preprocessing + 'yelp_user_features.csv', dtype=str)
df_yelp_item_feats = pd.read_csv(data_preprocessing + 'yelp_item_features.csv', dtype=str)
# df_yelp_inter

## NN-Based

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from deepctr.models import FNN, PNN, CCPM, NFM, WDL, DCN, DeepFM
from deepctr.models import AFM, xDeepFM, DIN
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [6]:
Kf = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_result = []

## NN-based

In [7]:
def get_feature_columns(df_all):
    # treat year as categorical
    sparse_features = list(df_all.columns[:-1])

    df_all[sparse_features] = df_all[sparse_features].fillna('-1')

    # 1. label encode
    for feat in sparse_features:
        lbe = LabelEncoder()
        df_all[feat] = lbe.fit_transform(df_all[feat])

    # 2. count #unique features for each sparse field
    fixlen_feature_columns = [SparseFeat(feat, df_all[feat].nunique()) for feat in sparse_features]
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
    return linear_feature_columns, dnn_feature_columns, feature_names

In [8]:
def calc_nn_evaluate(df_test, pred, k=10, threshold=3.5):
    rmse = round(np.sqrt(mean_squared_error(df_test['rating'].values, pred)), 4)
    dict_user_ratings = defaultdict(list)
    for i, x in df_test.iterrows():
        dict_user_ratings[x['user_id']].append((pred[i], x['rating']))
    
    ls_recall, ls_ndcg = [], []
    for user_id, ls_user_rating in dict_user_ratings.items():
        ls_user_rating.sort(key=lambda x: x[0], reverse=True)
        ls_true_sort = sorted(ls_user_rating, key=lambda x: x[1], reverse=True)
        # recall
        num_rel = sum(1 for (pred, true_rate) in ls_user_rating if true_rate > threshold)
        num_rel_topk = sum(1 for (pred, true_rate) in ls_user_rating[:k] if true_rate > threshold and pred > threshold)
        recall = num_rel_topk / num_rel if num_rel !=0 else 0
        ls_recall.append(recall)

        # ndcg
        dcg = sum(true_rate / math.log(i + 2, 2) for i, (pred, true_rate) in enumerate(ls_user_rating[:k]))
        idcg = sum(true_rate / math.log(i + 2, 2) for i, (pred, true_rate) in enumerate(ls_true_sort[:k]))
        ndcg = dcg / idcg
        ls_ndcg.append(ndcg)
    
    final_recall = round(sum(ls_recall) / len(ls_recall), 4)
    final_ndcg = round(sum(ls_ndcg) / len(ls_ndcg), 4)
    return [rmse, final_recall, final_ndcg]

In [9]:
def calc_nn_based_model(df_all, 
                        Model, model_params,
                        feature_names,
                        data_name, model_name):
    ls_result = []
    for i, (train_idx, test_idx) in enumerate(Kf.split(df_all)):
        df_train, df_test = df_all.iloc[train_idx].reset_index(drop=True), df_all.iloc[test_idx].reset_index(drop=True)
        train_model_input = {name: df_train[name] for name in feature_names}
        test_model_input = {name: df_test[name] for name in feature_names}
        model = Model(**model_params)
        model.compile('adam', 'mse', metrics=['mse'])
        es = EarlyStopping(monitor='val_mse', mode='min', patience=5, restore_best_weights=True)
        mdckpt = ModelCheckpoint(filepath=f'./saved_model/tf_{model_name}_{data_name}/', monitor='val_mse', save_best_only=True, mode='min')
        history = model.fit(
            train_model_input, df_train['rating'].values, 
            batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_split=0.2,
            callbacks=[es, mdckpt])
        pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
        print('test MSE', round(mean_squared_error(df_test['rating'].values, pred), 4))
        ls_result.append([model_name, i, data_name] + calc_nn_evaluate(df_test, pred))
    return ls_result

### douban

In [10]:
df_douban_all = pd.merge(df_douban_inter, df_douban_user_feats, on='user_id', how='left')
df_douban_all = pd.merge(df_douban_all, df_douban_item_feats, on='item_id', how='left')
df_douban_all['rating'] = df_douban_all.pop('rating')
print(df_douban_all.shape)
df_douban_all.head(3)

(790197, 7)


,user_id,item_id,location,author,publisher,year,rating
0,10855,938,33,4387,944,20,4
1,10027,3,394,9486,1743,14,3
2,741,2426,13,10548,1475,21,5


In [11]:
linear_feature_columns, dnn_feature_columns, feature_names = get_feature_columns(df_douban_all)
df_douban_all.head(3)

,user_id,item_id,location,author,publisher,year,rating
0,848,21659,244,4571,1755,13,4
1,30,14570,313,10236,828,6,3
2,9117,13934,32,612,530,14,5


In [12]:
# FNN
dict_FNN = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                FNN, dict_FNN,
                                feature_names, 
                                'douban', 'FNN')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.7376 - mse: 0.7363 - val_loss: 0.4997 - val_mse: 0.4972


Epoch 2/30
1976/1976 [==============================] - 13s 7ms/step - loss: 0.4811 - mse: 0.4774 - val_loss: 0.4973 - val_mse: 0.4925


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4677 - mse: 0.4616 - val_loss: 0.4948 - val_mse: 0.4877


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4537 - mse: 0.4454 - val_loss: 0.4976 - val_mse: 0.4884
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4404 - mse: 0.4301 - val_loss: 0.4973 - val_mse: 0.4861


Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4256 - mse: 0.4133 - val_loss: 0.5227 - val_mse: 0.5096
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4115 - mse: 0.3974 - val_loss: 0.5209 - val_mse: 0.5061
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3999 - mse: 0.3843 - val_loss: 0.5183 - val_mse: 0.5020
Epoch 9/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3903 - mse: 0.3732 - val_loss: 0.5287 - val_mse: 0.5111
Epoch 10/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3826 - mse: 0.3644 - val_loss: 0.5340 - val_mse: 0.5151
test MSE 0.4903
Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.7236 - mse: 0.7224 - val_loss: 0.5006 - val_mse: 0.4982


...

Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4625 - mse: 0.4563 - val_loss: 0.4996 - val_mse: 0.4924
Epoch 4/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4488 - mse: 0.4403 - val_loss: 0.4981 - val_mse: 0.4887
Epoch 5/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4356 - mse: 0.4251 - val_loss: 0.5009 - val_mse: 0.4895
Epoch 6/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4227 - mse: 0.4103 - val_loss: 0.5073 - val_mse: 0.4941
Epoch 7/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4109 - mse: 0.3968 - val_loss: 0.5138 - val_mse: 0.4989
test MSE 0.4842
Epoch 1/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.7006 - mse: 0.6994 - val_loss: 0.4985 - val_mse: 0.4961


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4810 - mse: 0.4773 - val_loss: 0.4936 - val_mse: 0.4889


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4632 - mse: 0.4571 - val_loss: 0.4931 - val_mse: 0.4860


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4493 - mse: 0.4409 - val_loss: 0.4916 - val_mse: 0.4822


Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4335 - mse: 0.4229 - val_loss: 0.4993 - val_mse: 0.4878
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4187 - mse: 0.4062 - val_loss: 0.5102 - val_mse: 0.4968
Epoch 7/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4060 - mse: 0.3917 - val_loss: 0.5181 - val_mse: 0.5030
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3955 - mse: 0.3796 - val_loss: 0.5272 - val_mse: 0.5106
Epoch 9/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3869 - mse: 0.3696 - val_loss: 0.5339 - val_mse: 0.5160
test MSE 0.4782


In [13]:
# IPNN
dict_IPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_inner': True}
ls_result = calc_nn_based_model(df_douban_all, 
                                PNN, dict_IPNN,
                                feature_names, 
                                'douban', 'IPNN')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.7071 - mse: 0.7060 - val_loss: 0.4965 - val_mse: 0.4944


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4682 - mse: 0.4650 - val_loss: 0.4852 - val_mse: 0.4810


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4475 - mse: 0.4421 - val_loss: 0.4852 - val_mse: 0.4787


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4262 - mse: 0.4186 - val_loss: 0.4996 - val_mse: 0.4910
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4076 - mse: 0.3980 - val_loss: 0.5037 - val_mse: 0.4931
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3941 - mse: 0.3826 - val_loss: 0.5158 - val_mse: 0.5035
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3841 - mse: 0.3710 - val_loss: 0.5221 - val_mse: 0.5083
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3769 - mse: 0.3624 - val_loss: 0.5326 - val_mse: 0.5174
test MSE 0.4833
Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.7258 - mse: 0.7248 - val_loss: 0.4948 - val_mse: 0.4927


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4724 - mse: 0.4693 - val_loss: 0.4874 - val_mse: 0.4833


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4511 - mse: 0.4458 - val_loss: 0.4882 - val_mse: 0.4819


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4330 - mse: 0.4256 - val_loss: 0.4926 - val_mse: 0.4843
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4158 - mse: 0.4063 - val_loss: 0.5044 - val_mse: 0.4940
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4017 - mse: 0.3904 - val_loss: 0.5168 - val_mse: 0.5046
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3901 - mse: 0.3772 - val_loss: 0.5195 - val_mse: 0.5058
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3817 - mse: 0.3673 - val_loss: 0.5286 - val_mse: 0.5135
test MSE 0.4823
Epoch 1/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.7138 - mse: 0.7128 - val_loss: 0.4947 - val_mse: 0.4926


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4657 - mse: 0.4624 - val_loss: 0.4876 - val_mse: 0.4832


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4389 - mse: 0.4333 - val_loss: 0.4937 - val_mse: 0.4870
Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4179 - mse: 0.4100 - val_loss: 0.5035 - val_mse: 0.4946
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4022 - mse: 0.3922 - val_loss: 0.5132 - val_mse: 0.5023
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3917 - mse: 0.3799 - val_loss: 0.5238 - val_mse: 0.5112
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3836 - mse: 0.3702 - val_loss: 0.5304 - val_mse: 0.5163
test MSE 0.4798


In [ ]:
# OPNN
dict_OPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_outter': True}
ls_result = calc_nn_based_model(df_douban_all, 
                                PNN, dict_OPNN,
                                feature_names, 
                                'douban', 'OPNN')
all_result.extend(ls_result)

1976/1976 [==============================] - 16s 7ms/step - loss: 0.7031 - mse: 0.7021 - val_loss: 0.4952 - val_mse: 0.4931


test MSE 0.4961
remember to take off break


In [ ]:
# PIN

In [ ]:
# CCPM
dict_CCPM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                CCPM, dict_CCPM,
                                feature_names, 
                                'douban', 'CCPM')
all_result.extend(ls_result)

1976/1976 [==============================] - 26s 12ms/step - loss: 0.7382 - mse: 0.7363 - val_loss: 0.5037 - val_mse: 0.5000


test MSE 0.5031
remember to take off break


In [ ]:
# WD
dict_WD = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                WDL, dict_WD,
                                feature_names, 
                                'douban', 'WD')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.7390 - mse: 0.7377 - val_loss: 0.4978 - val_mse: 0.4954


test MSE 0.4991
remember to take off break


In [ ]:
# DC
dict_DC = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                DCN, dict_DC,
                                feature_names, 
                                'douban', 'DC')
all_result.extend(ls_result)

CrossNet parameterization: vector
1976/1976 [==============================] - 17s 8ms/step - loss: 0.7164 - mse: 0.7151 - val_loss: 0.5063 - val_mse: 0.5038


test MSE 0.507
remember to take off break


In [ ]:
# NFM
dict_NFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                NFM, dict_NFM,
                                feature_names, 
                                'douban', 'NFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.7413 - mse: 0.7395 - val_loss: 0.4947 - val_mse: 0.4913


test MSE 0.4942
remember to take off break


In [ ]:
# deepFM
dict_deepFM = {'linear_feature_columns': linear_feature_columns, 
               'dnn_feature_columns': dnn_feature_columns,
               'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                DeepFM, dict_deepFM,
                                feature_names, 
                                'douban', 'deepFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.6969 - mse: 0.6957 - val_loss: 0.4999 - val_mse: 0.4975


test MSE 0.5005
remember to take off break


In [ ]:
# AFM
dict_AFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                AFM, dict_AFM,
                                feature_names, 
                                'douban', 'AFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 16s 7ms/step - loss: 2.5078 - mse: 2.5021 - val_loss: 0.6157 - val_mse: 0.6076


test MSE 0.6122
remember to take off break


In [ ]:
# xDeepFM
dict_xDeepFM = {'linear_feature_columns': linear_feature_columns,
                'dnn_feature_columns': dnn_feature_columns,
                'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                xDeepFM, dict_xDeepFM,
                                feature_names, 
                                'douban', 'xDeepFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 20s 9ms/step - loss: 0.6668 - mse: 0.6654 - val_loss: 0.4968 - val_mse: 0.4942


test MSE 0.4988
remember to take off break


In [ ]:
# DIN
# https://github.com/shenweichen/DeepCTR/blob/master/examples/run_din.py
# dict_DIN = {'dnn_feature_columns': dnn_feature_columns,
#             'history_feature_list': None,
#             'task': 'regression'}
# ls_result = calc_nn_based_model(df_douban_all, 
#                                 DIN, dict_DIN,
#                                 feature_names, 
#                                 'douban', 'DIN')
# all_result.extend(ls_result)

### movie lens

In [ ]:
df_movie_all = pd.merge(df_movie_inter, df_movie_user_feats, on='user_id', how='left')
df_movie_all = pd.merge(df_movie_all, df_movie_item_feats, on='item_id', how='left')
df_movie_all['rating'] = df_movie_all.pop('rating')
df_movie_all.iloc[:, :-1] = df_movie_all.iloc[:, :-1].fillna('0')
sr_movie_ts = df_movie_all.pop('timestamp')
print(df_movie_all.shape)
df_movie_all.head(3)

(100000, 23)


,user_id,item_id,age,occupation,genre1,genre10,genre11,genre12,genre13,genre14,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,196,242,5,3,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
1,186,302,4,4,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3
2,22,377,3,3,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [ ]:
linear_feature_columns, dnn_feature_columns, feature_names = get_feature_columns(df_movie_all)
df_movie_all.head(3)

,user_id,item_id,age,occupation,genre1,genre10,genre11,genre12,genre13,genre14,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,107,842,4,14,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
1,96,909,3,15,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3
2,134,991,2,14,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [ ]:
# FNN
dict_FNN = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                FNN, dict_FNN,
                                feature_names, 
                                'movie_lens', 'FNN')
all_result.extend(ls_result)

250/250 [==============================] - 8s 14ms/step - loss: 1.8399 - mse: 1.8398 - val_loss: 0.9044 - val_mse: 0.9044


test MSE 0.9163
remember to take off break


In [ ]:
# IPNN
dict_IPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_inner': True}
ls_result = calc_nn_based_model(df_movie_all, 
                                PNN, dict_IPNN,
                                feature_names, 
                                'movie_lens', 'IPNN')
all_result.extend(ls_result)

1976/1976 [==============================] - 14s 7ms/step - loss: 0.7286 - mse: 0.7275 - val_loss: 0.4997 - val_mse: 0.4977


test MSE 0.5015
remember to take off break


In [ ]:
# OPNN
dict_OPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_outter': True}
ls_result = calc_nn_based_model(df_movie_all, 
                                PNN, dict_OPNN,
                                feature_names, 
                                'movie_lens', 'OPNN')
all_result.extend(ls_result)

1976/1976 [==============================] - 16s 7ms/step - loss: 0.7031 - mse: 0.7021 - val_loss: 0.4952 - val_mse: 0.4931


test MSE 0.4961
remember to take off break


In [ ]:
# PIN

In [ ]:
# CCPM
dict_CCPM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                CCPM, dict_CCPM,
                                feature_names, 
                                'movie_lens', 'CCPM')
all_result.extend(ls_result)

1976/1976 [==============================] - 26s 12ms/step - loss: 0.7382 - mse: 0.7363 - val_loss: 0.5037 - val_mse: 0.5000


test MSE 0.5031
remember to take off break


In [ ]:
# WD
dict_WD = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                WDL, dict_WD,
                                feature_names, 
                                'movie_lens', 'WD')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.7390 - mse: 0.7377 - val_loss: 0.4978 - val_mse: 0.4954


test MSE 0.4991
remember to take off break


In [ ]:
# DC
dict_DC = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                DCN, dict_DC,
                                feature_names, 
                                'movie_lens', 'DC')
all_result.extend(ls_result)

CrossNet parameterization: vector
1976/1976 [==============================] - 17s 8ms/step - loss: 0.7164 - mse: 0.7151 - val_loss: 0.5063 - val_mse: 0.5038


test MSE 0.507
remember to take off break


In [ ]:
# NFM
dict_NFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                NFM, dict_NFM,
                                feature_names, 
                                'movie_lens', 'NFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.7413 - mse: 0.7395 - val_loss: 0.4947 - val_mse: 0.4913


test MSE 0.4942
remember to take off break


In [ ]:
# deepFM
dict_deepFM = {'linear_feature_columns': linear_feature_columns, 
               'dnn_feature_columns': dnn_feature_columns,
               'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                DeepFM, dict_deepFM,
                                feature_names, 
                                'movie_lens', 'deepFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.6969 - mse: 0.6957 - val_loss: 0.4999 - val_mse: 0.4975


test MSE 0.5005
remember to take off break


In [ ]:
# AFM
dict_AFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                AFM, dict_AFM,
                                feature_names, 
                                'movie_lens', 'AFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 16s 7ms/step - loss: 2.5078 - mse: 2.5021 - val_loss: 0.6157 - val_mse: 0.6076


test MSE 0.6122
remember to take off break


In [ ]:
# xDeepFM
dict_xDeepFM = {'linear_feature_columns': linear_feature_columns,
                'dnn_feature_columns': dnn_feature_columns,
                'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                xDeepFM, dict_xDeepFM,
                                feature_names, 
                                'movie_lens', 'xDeepFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 20s 9ms/step - loss: 0.6668 - mse: 0.6654 - val_loss: 0.4968 - val_mse: 0.4942


test MSE 0.4988
remember to take off break


### yelp

In [ ]:
df_yelp_all = pd.merge(df_yelp_inter, df_yelp_user_feats, on='user_id', how='left')
df_yelp_all = pd.merge(df_yelp_all, df_movie_item_feats, on='item_id', how='left')
df_yelp_all['rating'] = df_yelp_all.pop('rating')
df_yelp_all.iloc[:, :-1] = df_yelp_all.iloc[:, :-1].fillna('0')
print(df_yelp_all.shape)
df_yelp_all.head(3)

(188456, 32)


,user_id,item_id,compliment1,compliment10,compliment11,compliment2,compliment3,compliment4,compliment5,compliment6,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,2,186,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,5
1,2,205,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,5
2,2,209,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,1,0,4


In [ ]:
linear_feature_columns, dnn_feature_columns, feature_names = get_feature_columns(df_yelp_all)
df_yelp_all.head(3)

,user_id,item_id,compliment1,compliment10,compliment11,compliment2,compliment3,compliment4,compliment5,compliment6,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,3867,5209,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,5
1,3867,5418,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,5
2,3867,5461,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,1,0,4


In [ ]:
# FNN
dict_FNN = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                FNN, dict_FNN,
                                feature_names, 
                                'yelp', 'FNN')
all_result.extend(ls_result)

250/250 [==============================] - 8s 14ms/step - loss: 1.8399 - mse: 1.8398 - val_loss: 0.9044 - val_mse: 0.9044


test MSE 0.9163
remember to take off break


In [ ]:
# IPNN
dict_IPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_inner': True}
ls_result = calc_nn_based_model(df_yelp_all, 
                                PNN, dict_IPNN,
                                feature_names, 
                                'yelp', 'IPNN')
all_result.extend(ls_result)

1976/1976 [==============================] - 14s 7ms/step - loss: 0.7286 - mse: 0.7275 - val_loss: 0.4997 - val_mse: 0.4977


test MSE 0.5015
remember to take off break


In [ ]:
# OPNN
dict_OPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_outter': True}
ls_result = calc_nn_based_model(df_yelp_all, 
                                PNN, dict_OPNN,
                                feature_names, 
                                'yelp', 'OPNN')
all_result.extend(ls_result)

1976/1976 [==============================] - 16s 7ms/step - loss: 0.7031 - mse: 0.7021 - val_loss: 0.4952 - val_mse: 0.4931


test MSE 0.4961
remember to take off break


In [ ]:
# PIN

In [ ]:
# CCPM
dict_CCPM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                CCPM, dict_CCPM,
                                feature_names, 
                                'yelp', 'CCPM')
all_result.extend(ls_result)

1976/1976 [==============================] - 26s 12ms/step - loss: 0.7382 - mse: 0.7363 - val_loss: 0.5037 - val_mse: 0.5000


test MSE 0.5031
remember to take off break


In [ ]:
# WD
dict_WD = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                WDL, dict_WD,
                                feature_names, 
                                'yelp', 'WD')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.7390 - mse: 0.7377 - val_loss: 0.4978 - val_mse: 0.4954


test MSE 0.4991
remember to take off break


In [ ]:
# DC
dict_DC = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                DCN, dict_DC,
                                feature_names, 
                                'yelp', 'DC')
all_result.extend(ls_result)

CrossNet parameterization: vector
1976/1976 [==============================] - 17s 8ms/step - loss: 0.7164 - mse: 0.7151 - val_loss: 0.5063 - val_mse: 0.5038


test MSE 0.507
remember to take off break


In [ ]:
# NFM
dict_NFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                NFM, dict_NFM,
                                feature_names, 
                                'yelp', 'NFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.7413 - mse: 0.7395 - val_loss: 0.4947 - val_mse: 0.4913


test MSE 0.4942
remember to take off break


In [ ]:
# deepFM
dict_deepFM = {'linear_feature_columns': linear_feature_columns, 
               'dnn_feature_columns': dnn_feature_columns,
               'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                DeepFM, dict_deepFM,
                                feature_names, 
                                'yelp', 'deepFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 15s 7ms/step - loss: 0.6969 - mse: 0.6957 - val_loss: 0.4999 - val_mse: 0.4975


test MSE 0.5005
remember to take off break


In [ ]:
# AFM
dict_AFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                AFM, dict_AFM,
                                feature_names, 
                                'yelp', 'AFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 16s 7ms/step - loss: 2.5078 - mse: 2.5021 - val_loss: 0.6157 - val_mse: 0.6076


test MSE 0.6122
remember to take off break


In [ ]:
# xDeepFM
dict_xDeepFM = {'linear_feature_columns': linear_feature_columns,
                'dnn_feature_columns': dnn_feature_columns,
                'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                xDeepFM, dict_xDeepFM,
                                feature_names, 
                                'yelp', 'xDeepFM')
all_result.extend(ls_result)

1976/1976 [==============================] - 20s 9ms/step - loss: 0.6668 - mse: 0.6654 - val_loss: 0.4968 - val_mse: 0.4942


test MSE 0.4988
remember to take off break
